In [4]:
%load_ext autoreload
%autoreload 2

import os,sys,warnings
import h5py, pandas
import numpy as np
np.seterr(invalid=['ignore','warn'][0])
import math,cmath,pickle
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit,fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('default')
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.titlesize'] = 20
mpl.rcParams['figure.figsize'] = [6.4*1.2,4.8*1.2]
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 30
mpl.rcParams['lines.marker'] = 's'
mpl.rcParams['lines.linestyle'] = ''
mpl.rcParams['lines.markersize'] = 12
mpl.rcParams['errorbar.capsize'] = 12
mpl.rcParams['xtick.labelsize'] = mpl.rcParams['ytick.labelsize'] = 22
mpl.rcParams['xtick.major.size'] = mpl.rcParams['ytick.major.size'] = 10
mpl.rcParams['xtick.top']=mpl.rcParams['ytick.right']=True
mpl.rcParams['xtick.direction']=mpl.rcParams['ytick.direction']='in'
mpl.rcParams['legend.fontsize'] = 24
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

# mpl.rcParams.update({"axes.grid" : True})
import util as yu
yu.flag_fast=False


enss=['a24','a']
# enss=['a','b']
ens2full={'a24':'cA211.53.24','a':'cA2.09.48','b':'B64','c':'C80','d':'D96'}
ens2label={'a24':'A24','a':'A48','b':'B64','c':'C80','d':'D96'}
ens2a={'a24':0.0908,'a':0.0938,'b':0.07957,'c':0.06821,'d':0.05692} # fm
ens2N={'a24':24,'a':48,'b':64,'c':80,'d':96}
ens2N_T={'a24':24*2,'a':48*2,'b':64*2,'c':80*2,'d':96*2}

ens2mN={'a':931}
ens2mpiC={'a':131}
ens2mpi0={'a':111}

hbarc = 1/197.3
ens2aInv={ens:1/(ens2a[ens]*hbarc) for ens in enss} # MeV

def fsolve2(func,x0):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        res=fsolve(func, x0)[0]
    return res if res!=x0 else np.NaN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
Tpack=24
path='/project/s1174/lyan/code/projectData/NST_f/data/NST_f_cA2.09.48_scattering12.h5'
data=yu.load(path,d=6)
Njk=yu.deepKey(data['2pt'],2).shape[0]

loading: /project/s1174/lyan/code/projectData/NST_f/data/NST_f_cA2.09.48_scattering12.h5
91/91: diags/pi0f-pi0i/opabs                                                        
Ncfg=1228, Njk=204


In [3]:
flags={
    'cc2pt':True, # average quantities related by complex conjugation for 2pt
    'cc3pt':True, # same for 3pt (Removal of vacuum expectation value requires 'cc2pt'=='cc3pt')
    'll2pt':True, # average irrep rows 'l1' and conjugated 'l2' (Parity breaking effect of tmQCD has been taken care of)
    'll3pt':True, # same for 3pt (This flag has no effect if 'll2pt'=True and spin-projection is done)
    'r2pt': True, # making 2ptMat real for the rest frame # 'll2pt' has to be real for this flag
    'remove_pi0VEV':True, 
}

def op_remove_pi0(op):
    t=op.split(';')
    if t[-1] in ['p','n','n,pi+']:
        return False
    if t[3] not in ['N0pi0,a','N1pi0,a']:
        return False
    if t[2] == 'G1u':
        t[2]='G1g'
        assert(t[3]=='N0pi0,a'); t[3]='a'
        assert(t[-1]=='p,pi0'); t[-1]='p'
        return ';'.join(t)

def get2pt_diag(opa,opb,diag):
    opab=f'{opa}_{opb}'
    if opab not in data['2pt'].keys() or diag not in data['2pt'][opab].keys():
        return 0
    res=data['2pt'][opab][diag].copy()
    if not flags['remove_pi0VEV']:
        return res
    if diag == 'pi0f-pi0i':
        if opa==opb and opa=='t;0,0,0;pi0':
            res -= (data['VEV']['pi0f']**2)[:,None]
        return res
    
    if 'pi0f' in diag:
        t_opa=op_remove_pi0(opa)
        if t_opa != False:
            t=diag.split('-'); t.remove('pi0f'); t_diag='-'.join(t)
            res -= data['2pt'][f'{t_opa}_{opb}'][t_diag] * data['VEV']['pi0f'][:,None]
    if 'pi0i' in diag:
        t_opb=op_remove_pi0(opb)
        if t_opb != False:
            t=diag.split('-'); t.remove('pi0i'); t_diag='-'.join(t)
            res -= data['2pt'][f'{opa}_{t_opb}'][t_diag] * np.conj(data['VEV']['pi0f'][:,None])
    if 'pi0f' in diag and 'pi0i' in diag:
        t_opa=op_remove_pi0(opa); t_opb=op_remove_pi0(opb)
        if t_opa != False and t_opb != False:
            t=diag.split('-'); t.remove('pi0f'); t.remove('pi0i'); t_diag='-'.join(t)
            res += data['2pt'][f'{t_opa}_{t_opb}'][t_diag] * data['VEV']['pi0f'][:,None] * np.conj(data['VEV']['pi0f'][:,None])
    return res

def get2pt(opa,opb,diags=yu.diags_all):
    if opa.split(';')[-1]=='12':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa1,opb,diags)-np.sqrt(1/3)*get2pt(opa2,opb,diags)
    elif opa.split(';')[-1]=='32':
        opa1=';'.join(opa.split(';')[:-1]+['n,pi+']); opa2=';'.join(opa.split(';')[:-1]+['p,pi0'])
        return np.sqrt(1/3)*get2pt(opa1,opb,diags)+np.sqrt(2/3)*get2pt(opa2,opb,diags)
    if opb.split(';')[-1]=='12':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(2/3)*get2pt(opa,opb1,diags)-np.sqrt(1/3)*get2pt(opa,opb2,diags)
    elif opb.split(';')[-1]=='32':
        opb1=';'.join(opb.split(';')[:-1]+['n,pi+']); opb2=';'.join(opb.split(';')[:-1]+['p,pi0'])
        return np.sqrt(1/3)*get2pt(opa,opb1,diags)+np.sqrt(2/3)*get2pt(opa,opb2,diags)
    
    res=0
    opab=f'{opa}_{opb}'; opba=f'{opb}_{opa}'
    if opab in data['2pt']:
        res+=np.sum([get2pt_diag(opa,opb,diag) for diag in data['2pt'][opab].keys() if diag in diags],axis=0)
    if opba in data['2pt']:
        diags_cc={'T', 'T-pi0f'}; 
        res+=np.conj(np.sum([get2pt_diag(opa,opb,diag) for diag in data['2pt'][opba].keys() if diag in diags_cc and diag in diags],axis=0))
    return res

def get2ptMat(ops,diags=yu.diags_all):
    if flags['ll2pt']:
        flags['ll2pt']=False
        ops_flip=[yu.op_flipl(op) for op in ops]
        t=(get2ptMat(ops,diags=diags)+np.conj(get2ptMat(ops_flip,diags=diags)))/2
        flags['ll2pt']=True
        if ops[0].split(';')[1]=='0,0,0' and flags['r2pt']:
            t=np.real(t)
        return t
    t=np.transpose([[get2pt(opa,opb,diags) for opb in ops] for opa in ops],[2,3,0,1])
    if flags['cc2pt']:
        t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t


In [23]:
# sigma term to lattice scattering length 1/2 # Hoferichter:2016ocj
mpiC_phys=139.57
sgm_center=59.1
a12_center=169.8*1e-3 * (mpiC_phys**-1); a12_error=2.0*1e-3 * (mpiC_phys**-1)
a32_center=-86.3*1e-3 * (mpiC_phys**-1); a32_error=1.8*1e-3 * (mpiC_phys**-1)
a12_center_pheLat=178.8*1e-3 * (mpiC_phys**-1); a12_error_pheLat=3.8*1e-3 * (mpiC_phys**-1)
a32_center_pheLat=-77.5*1e-3 * (mpiC_phys**-1); a32_error_pheLat=3.5*1e-3 * (mpiC_phys**-1)

def a2sgm(a12,a32):
    c12=0.242*1e3*mpiC_phys
    c32=0.874*1e3*mpiC_phys
    return sgm_center + c12*(a12-a12_center) + c32*(a32-a32_center)

def sgm2a12(sgm):    
    c12=0.242*1e3*mpiC_phys
    c32=0.874*1e3*mpiC_phys
    return a12_center + (sgm-sgm_center)/c12

# scattering length <-> energy shift
def sl2es(mu, L, a0):
    c1=-2.837297; c2=6.375183
    return - (2*np.pi)/(mu*L) * a0/L * ( 1 + c1*a0/L + c2*(a0/L)**2 ) / L
def es2sl(mu, L, dE):
    return fsolve2(lambda a0:sl2es(mu, L, a0)-dE, 0)

mN=931; mpi=131
mu=1/(1/mN+1/mpi); L=4.5 * hbarc

print(f'1/2: {sl2es(mu,L,a12_center)}, 3/2: {sl2es(mu,L,a32_center)}')

for sgm in [40,60]:
    print(f'{sgm}: {sl2es(mu,L,sgm2a12(sgm))}')
    
es2sl(mu,L,-5),a12_center

# a2sgm(0.169*mpiC_phys**-1/2,-0.1*mpiC_phys**-1)

1/2: -4.862830827012197, 3/2: 3.0840020416562544
40: -2.774864258196807
60: -4.954939472888397


(np.float64(0.001256330327401107), 0.0012165938238876551)

In [10]:
diags=yu.diags_all
# diags=yu.diags_pi0Loopless

func_c2pt2meff=lambda c2pt: np.log(c2pt/np.roll(c2pt,-1,axis=0))

fig, axs = yu.getFigAxs(4,3,scale=1.5,sharex=True,sharey='row')
ens='a'
xunit=ens2a[ens]; yunit=ens2aInv[ens]

ax=axs[0,0]
tmin,tmax=5,16

op='g;0,0,0;G1u;N0pi0,a;l1;12'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='r',label=r'$E_{N\pi[1/2]}$')

# op='g;0,0,0;G1u;N0pi0,a;l1;32'
# c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0])
# meff=yu.jackmap(func_c2pt2meff,c2pt)
# meff_mean,meff_err=yu.jackme(meff)
# times=np.arange(len(meff_mean))

# plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
# ax.errorbar(plt_x,plt_y,plt_err,color='purple',label=r'$E_{N\pi[3/2]}$')

op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi+'
c2pt=np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='b',label=r'$E^{\rm free}_{n\,\pi^+}$')

op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi0'
c2pt=np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='orange',label=r'$E^{\rm free}_{p\,\pi^0}$')

ax.legend()

ax=axs[0,1]

op='g;0,0,0;G1u;N0pi0,a;l1;n,pi+'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='r',label=r'$E_{n\,\pi^+}$')

op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi+'
c2pt=np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='b',label=r'$E^{\rm free}_{n\,\pi^+}$')

ax.legend()

ax=axs[0,2]

op='g;0,0,0;G1u;N0pi0,a;l1;p,pi0'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='r',label=r'$E_{p\,\pi^0}$')

op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi0'
c2pt=np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))
# tmin,tmax=1,17

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='orange',label=r'$E^{\rm free}_{p\,\pi^0}$')

ax.legend()

#============================================================================

ax=axs[1,0]
tmin,tmax=1,16

op='g;0,0,0;G1u;N0pi0,a;l1;12'; op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi+'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='b',label=r'$E_{N\pi[1/2]} - E^{\rm free}_{n\,\pi^+}$')

op='g;0,0,0;G1u;N0pi0,a;l1;12'; op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi0'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='orange',label=r'$E_{N\pi[1/2]} - E^{\rm free}_{p\,\pi^0}$')

op_piP='t;0,0,0;pi+'; op_pi0='t;0,0,0;pi0'
c2pt=np.real(get2pt(op_piP,op_piP,diags=diags)[:,:Tpack]) / np.real(get2pt(op_pi0,op_pi0,diags=diags)[:,:Tpack])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='g',label=r'$E^{\rm free}_{\pi^+} - E^{\rm free}_{\pi^0}$')

ax.axhline(0,color='grey',linestyle='--')
ax.axhline(sl2es(mu,L,a12_center),color='r',linestyle='dotted',label='pheno')
ax.set_ylim(-40,25)
ax.legend()


ax=axs[1,1]

op='g;0,0,0;G1u;N0pi0,a;l1;n,pi+'; op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi+'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='r',label=r'$E_{n\,\pi^+} - E^{\rm free}_{n\,\pi^+}$')

ax.legend()
ax.axhline(0,color='grey',linestyle='--')

ax=axs[1,2]

op='g;0,0,0;G1u;N0pi0,a;l1;p,pi0'; op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi0'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='r',label=r'$E_{p\,\pi^0} - E^{\rm free}_{p\,\pi^0}$')

ax.legend()
ax.axhline(0,color='grey',linestyle='--')

#============================================================================

ax=axs[2,0]
tmin,tmax=1,16

op='g;0,0,0;G1u;N0pi0,a;l1;32'; op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi+'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='b',label=r'$E_{N\pi[3/2]} - E^{\rm free}_{n\,\pi^+}$')

op='g;0,0,0;G1u;N0pi0,a;l1;32'; op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi0'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='orange',label=r'$E_{N\pi[3/2]} - E^{\rm free}_{p\,\pi^0}$')

op_piP='t;0,0,0;pi+'; op_pi0='t;0,0,0;pi0'
c2pt=np.real(get2pt(op_piP,op_piP,diags=diags)[:,:Tpack]) / np.real(get2pt(op_pi0,op_pi0,diags=diags)[:,:Tpack])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='g',label=r'$E^{\rm free}_{\pi^+} - E^{\rm free}_{\pi^0}$')

ax.axhline(0,color='grey',linestyle='--')
ax.axhline(sl2es(mu,L,a32_center),color='r',linestyle='dotted',label='pheno')
ax.set_ylim(-40,25)
ax.legend()

#============================================================================

ax=axs[3,0]
tmin,tmax=1,16

op32='g;0,0,0;G1u;N0pi0,a;l1;32'; op12='g;0,0,0;G1u;N0pi0,a;l1;12'
c2pt=np.real(get2ptMat([op32],diags=diags)[:,:,0,0]) / np.real(get2ptMat([op12],diags=diags)[:,:,0,0])
meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='b',label=r'$E_{N\pi[3/2]} - E_{N\pi[1/2]}$')

ax.axhline(0,color='grey',linestyle='--')
ax.axhline(sl2es(mu,L,a32_center)-sl2es(mu,L,a12_center),color='r',linestyle='dotted',label='pheno')
ax.legend()

plt.tight_layout()
plt.savefig('fig/scattering.pdf')
plt.close()

In [6]:
relaThreshold=0.2

diags=yu.diags_all
# diags=yu.diags_pi0Loopless

func_c2pt2meff=lambda c2pt: np.log(c2pt/np.roll(c2pt,-1,axis=0))

fig, axs = yu.getFigAxs(4,2,scale=1.5,sharex='row',sharey='row')
ens='a'
xunit=ens2a[ens]; yunit=ens2aInv[ens]
ylim_dE=[-30,30]

iso='12'
iso_str=iso[0]+'/'+iso[1]
ax=axs[0,0]
tmin,tmax=1,16
fitmax=tmax
color='r'

op=f'g;0,0,0;G1u;N0pi0,a;l1;{iso}'; op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi+'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )

meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color=color,label=r'$E_{N\pi\,['+iso_str+r']} - E^{\rm free}_{n\,\pi^+}$')

ax.set_ylim(ylim_dE)
ax.axhline(0,color='grey',linestyle='--')
ax.legend()

ax=axs[0,1]

fitmins=np.arange(1,15); fits=[]
for fitmin in fitmins:
    fit_t=np.arange(fitmin,fitmax)
    fitfunc=lambda dE: np.zeros(len(fit_t))+dE
    pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,meff[:,fit_t],[0])
    fits.append([pars_jk,chi2_jk,Ndof])
    
    pars_mean,pars_err=yu.jackme(pars_jk)
    chi2_mean,chi2_err=yu.jackme(chi2_jk)
    chi2R=chi2_mean[0]/Ndof
    
    plt_x=np.array([fitmin])*xunit; plt_y=np.array([pars_mean[0]])*yunit; plt_err=np.array([pars_err[0]])*yunit
    ax.errorbar(plt_x,plt_y,plt_err,color=color)
    ylim=ax.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/20
    ax.annotate('{:.2f}'.format(chi2R),(plt_x,plt_y-plt_err-chi2_shift),color=color,ha='center')

pars_jk,props_jk=yu.jackMA(fits)
pars_jk1=pars_jk
pars_mean,pars_err=yu.jackme(pars_jk)
plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[0]*yunit; plt_error=pars_err[0]*yunit
ax.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2)
    
ax.axhline(0,color='grey',linestyle='--')


ax=axs[1,0]
tmin,tmax=1,16
fitmax=tmax
color='b'

op_N='g;0,0,0;G1g;a;l1;p'; op_pi='t;0,0,0;pi0'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*np.real(get2pt(op_pi,op_pi,diags=diags)[:,:Tpack]) )

meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color=color,label=r'$E_{N\pi\,['+iso_str+r']} - E^{\rm free}_{p\,\pi^0}$')

ax.set_ylim(ylim_dE)
ax.axhline(0,color='grey',linestyle='--')
ax.legend()

ax=axs[1,1]

fitmins=np.arange(1,15); fits=[]
for fitmin in fitmins:
    fit_t=np.arange(fitmin,fitmax)
    fitfunc=lambda dE: np.zeros(len(fit_t))+dE
    pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,meff[:,fit_t],[0])
    fits.append([pars_jk,chi2_jk,Ndof])
    
    pars_mean,pars_err=yu.jackme(pars_jk)
    chi2_mean,chi2_err=yu.jackme(chi2_jk)
    chi2R=chi2_mean[0]/Ndof
    
    plt_x=np.array([fitmin])*xunit; plt_y=np.array([pars_mean[0]])*yunit; plt_err=np.array([pars_err[0]])*yunit
    ax.errorbar(plt_x,plt_y,plt_err,color=color)
    ylim=ax.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/20
    ax.annotate('{:.2f}'.format(chi2R),(plt_x,plt_y-plt_err-chi2_shift),color=color,ha='center')

pars_jk,props_jk=yu.jackMA(fits)
pars_jk2=pars_jk
pars_mean,pars_err=yu.jackme(pars_jk)
plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[0]*yunit; plt_error=pars_err[0]*yunit
ax.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2)
    
ax.axhline(0,color='grey',linestyle='--')

ax=axs[2,0]
tmin,tmax=1,16
fitmax=tmax
color='g'

op_N='g;0,0,0;G1g;a;l1;p'; op_piC='t;0,0,0;pi+'; op_pi0='t;0,0,0;pi0'
c2pt=np.real(get2ptMat([op],diags=diags)[:,:,0,0]) / ( np.real(get2ptMat([op_N],diags=diags)[:,:,0,0])*(np.real(get2pt(op_piC,op_piC,diags=diags)[:,:Tpack])+np.real(get2pt(op_pi0,op_pi0,diags=diags)[:,:Tpack]))/2 )

meff=yu.jackmap(func_c2pt2meff,c2pt)
meff_mean,meff_err=yu.jackme(meff)
times=np.arange(len(meff_mean))

plt_x=times[tmin:tmax]*xunit; plt_y=meff_mean[tmin:tmax]*yunit; plt_err=meff_err[tmin:tmax]*yunit
ax.errorbar(plt_x,plt_y,plt_err,color=color,label=r'$E_{N\pi\,['+iso_str+r']} - E^{\rm free}_{(n\,\pi^+ + p\,\pi^0)/2}$')

ax.set_ylim(ylim_dE)
ax.axhline(0,color='grey',linestyle='--')
ax.legend()

ax=axs[2,1]

fitmins=np.arange(1,15); fits=[]
for fitmin in fitmins:
    fit_t=np.arange(fitmin,fitmax)
    fitfunc=lambda dE: np.zeros(len(fit_t))+dE
    pars_jk,chi2_jk,Ndof=yu.jackfit(fitfunc,meff[:,fit_t],[0])
    fits.append([pars_jk,chi2_jk,Ndof])
    
    pars_mean,pars_err=yu.jackme(pars_jk)
    chi2_mean,chi2_err=yu.jackme(chi2_jk)
    chi2R=chi2_mean[0]/Ndof
    
    plt_x=np.array([fitmin])*xunit; plt_y=np.array([pars_mean[0]])*yunit; plt_err=np.array([pars_err[0]])*yunit
    ax.errorbar(plt_x,plt_y,plt_err,color=color)
    ylim=ax.get_ylim(); chi2_shift=(ylim[1]-ylim[0])/20
    ax.annotate('{:.2f}'.format(chi2R),(plt_x,plt_y-plt_err-chi2_shift),color=color,ha='center')

pars_jk,props_jk=yu.jackMA(fits)
pars_jk3=pars_jk
pars_mean,pars_err=yu.jackme(pars_jk)
plt_x=np.array([fitmins[0]-0.5,fitmins[-1]+0.5])*xunit; plt_y=pars_mean[0]*yunit; plt_error=pars_err[0]*yunit
ax.fill_between(plt_x,plt_y-plt_error,plt_y+plt_error,color=color,alpha=0.2)
    
ax.axhline(0,color='grey',linestyle='--')

ax=axs[3,1]
xunit=1

# mpi=mpiC_phys
# sl_mean=[a12_center_pheLat]; sl_err=[a12_error_pheLat]
# yunit=mpi
# plt_x=np.array([0])*xunit; plt_y=np.array([sl_mean[0]])*yunit; plt_err=np.array([sl_err[0]])*yunit
# ax.errorbar(plt_x,plt_y,plt_err,color='black',mfc='white')

mpi=mpiC_phys
sl_mean=[a12_center]; sl_err=[a12_error]
yunit=mpi
plt_x=np.array([1])*xunit; plt_y=np.array([sl_mean[0]])*yunit; plt_err=np.array([sl_err[0]])*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='black')

# piC
mN=ens2mN['a']; mpi=ens2mpiC['a']
mu=1/(1/mN+1/mpi); L=ens2a['a']*ens2N['a'] * hbarc
func=lambda dE:[es2sl(mu,L,dE)]
sl=yu.jackmap(func,pars_jk1)
sl_mean,sl_err=yu.jackme(sl)
yunit=ens2aInv['a']*mpi
plt_x=np.array([2])*xunit; plt_y=np.array([sl_mean[0]])*yunit; plt_err=np.array([sl_err[0]])*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='r')

# pi0
mN=ens2mN['a']; mpi=ens2mpi0['a']
mu=1/(1/mN+1/mpi); L=ens2a['a']*ens2N['a'] * hbarc
func=lambda dE:[es2sl(mu,L,dE)]
sl=yu.jackmap(func,pars_jk2)
sl_mean,sl_err=yu.jackme(sl)
yunit=ens2aInv['a']*mpi
plt_x=np.array([3])*xunit; plt_y=np.array([sl_mean[0]])*yunit; plt_err=np.array([sl_err[0]])*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='b')

# avg
mN=ens2mN['a']; mpi=(ens2mpiC['a']+ens2mpi0['a'])/2
mu=1/(1/mN+1/mpi); L=ens2a['a']*ens2N['a'] * hbarc
func=lambda dE:[es2sl(mu,L,dE)]
sl=yu.jackmap(func,pars_jk3)
sl_mean,sl_err=yu.jackme(sl)
yunit=ens2aInv['a']*mpi
plt_x=np.array([4])*xunit; plt_y=np.array([sl_mean[0]])*yunit; plt_err=np.array([sl_err[0]])*yunit
ax.errorbar(plt_x,plt_y,plt_err,color='g')

ax.set_xticks([1,2,3,4],['exp',r'$n\,\pi^+$',r'$p\,\pi^0$',r'avg'])
ax.axhline(0,color='grey',linestyle='--')

plt.tight_layout()
plt.savefig('fig/scattering12_fit.pdf')
plt.close()
